In [4]:
%matplotlib inline

In [5]:
import itertools
import pickle
from collections import defaultdict
import numpy as np
from contextlib import closing
import numpy.ma as ma
import pickle
import pandas as pd
import sqlalchemy
import multiprocessing
import csv
import os
import sqlite3
import time
import dill
from tqdm import tqdm
from glob import iglob

### Load in dictionary of probabilities

In [6]:
with open("/data1/kji/databases/probabilities.pkl", "rb") as f:
    utilities = pickle.load(f)

### Load in dictionary of n-grams and their counts

In [7]:
db_dir = '/data1/kji/databases'

In [5]:
files = []
for filename in iglob(f"{db_dir}/*_counts.pkl", recursive=True):
    files.append(filename)

In [6]:
files.sort(key = len)

In [8]:
def marginal_memory(num_matches, remaining_budget):
    return num_matches / remaining_budget

In [9]:
def marginal_runtime(matches_squared, num_matches, total_matches, total_avg_runtime, runtime_budget):
    cost = (matches_squared + num_matches ** 2) / (total_matches + num_matches) - total_avg_runtime
    return cost / (runtime_budget - total_avg_runtime)

In [10]:
def metric(value_pair, memory_budget, runtime_budget, total_matches, matches_squared, total_avg_runtime):
    print(value_pair)
    utility = value_pair[0]
    num_matches = value_pair[1]
    marginal_memory_cost = marginal_memory(num_matches, memory_budget-total_matches)
    marginal_runtime_cost = marginal_runtime(matches_squared, num_matches, total_matches, total_avg_runtime, runtime_budget)
    return utility / max(marginal_memory_cost, marginal_runtime_cost)

In [14]:
def store_DB(db, combination, outdir):
    with open(f"{outdir}/{combination}.pkl", "wb") as f:
        pickle.dump(db, f)

## Test database with in memory dictionary

First work with a subset of the data for prototyping.

In [11]:
test_files = ['/data1/kji/databases/0_counts.pkl', 
              '/data1/kji/databases/01_counts.pkl',
              '/data1/kji/databases/012_counts.pkl',
              '/data1/kji/databases/0123_counts.pkl']

Store each n-gram, utility, and the number of matches for an n-gram in a separate array.

In [ ]:
n_grams = np.array([], dtype = object)
probabilities = np.array([], dtype = float)
matches = np.array([], dtype = int)

In [13]:
for filename in test_files:
    with open(filename, "rb") as f:
        d = pickle.load(f)
        combination = filename.split('/')[-1].split("_")[0]
        length = len(d)
        fps = np.empty(length, dtype = object)
        utils = np.full(length, utilities[combination])
        values = np.empty(length, dtype = int)
        for i, (n_gram, count) in enumerate(d.items()):
            fps[i] = n_gram
            values[i] = count
        d.clear()
        n_grams = np.concatenate([n_grams, fps])
        probabilities = np.concatenate([probabilities, utils])
        matches = np.concatenate([matches, values])
        print(f"finished processing {filename}")
        f.flush()

finished processing /data1/kji/databases/0_counts.pkl
finished processing /data1/kji/databases/01_counts.pkl
finished processing /data1/kji/databases/012_counts.pkl
finished processing /data1/kji/databases/0123_counts.pkl


In [15]:
store_DB(n_grams, "fps_1-4", '/data1/kji/databases')

In [16]:
store_DB(probabilities, "utils_1-4", '/data1/kji/databases')

In [17]:
store_DB(matches, "matches_1-4", '/data1/kji/databases')

In [ ]:
with open('/data1/kji/databases/fps_1-4/pkl', "rb") as f:
    n_grams = pickle.load(f)
    
with open('/data1/kji/databases/utils_1-4/pkl', "rb") as f:
    probabilities = pickle.load(f)
    
with open('/data1/kji/databases/matches_1-4/pkl', "rb") as f:
    matches = pickle.load(f)

In [44]:
def construct_db(Bm, Bt, outdir, outfile_name):
    """
    Inputs: a memory budget and a runtime budget
    Output: a file specifying an ordered list of n-grams to include in the final database
    """
    # total number of matches for all fingerprints in the database
    Dm = 0
    Dm_squared = 0
    # cumulative average runtime cost of all fingerprints in the database
    Dt = 0
    mask = np.zeros(n_grams.shape)
    with open(f"{outdir}/{outfile_name}.txt", "w") as f1, open(f"{outdir}/{outfile_name}.csv", "w") as f2:
        writer = csv.writer(f2)
        writer.writerow(["fingerprint", "ratio", "m_i", "Ct_i", "Bm-Dm", "Bt-Dt"])
        while Bm - Dm > 0 and Bt - Dt > 0:
            start = time.time()
            f = lambda x: x / (Bm-Dm)
            g = lambda x: ((x ** 2 + Dm_squared) / (x + Dm) - Dt) / (Bt - Dt)
            # vectorize computation of marginal costs
            marginal_memory = f(matches)
            marginal_runtime = g(matches)
            ratios = probabilities / np.maximum(marginal_memory, marginal_runtime)
            # use a mask array to keep track of which fingerprints we've already selected
            x = ma.array(ratios, mask = mask)
            # get the fingerprint with the highest utility:cost ratio
            idx = np.argmax(x)
            result = n_grams[idx]
            mask[idx] = 1
            f1.write(f"{result}\n")
            
            ratio = ratios[idx]
            m_i = matches[idx]
            
            Dm += m_i
            Dm_squared += m_i**2
            
            cost = Dm_squared / Dm - Dt
            Dt = Dm_squared / Dm
            
            remaining_memory = Bm - Dm
            remaining_runtime = Bt - Dt
            writer.writerow([result, ratio, m_i, cost, remaining_memory, remaining_runtime])
#             print(f"finished in {time.time() - start} seconds")

In [45]:
memory_budget = 10000
runtime_budget = 10000

In [ ]:
construct_db(memory_budget, runtime_budget, "experiments/db_tests", "numpy_test")

In [ ]:
construct_db(100000, 100000, "experiments/db_tests", "numpy_test_100k")

## SQL database

In [5]:
engine = sqlalchemy.create_engine('sqlite:////data1/kji/databases/test.db')

In [12]:
test_files = ['/data1/kji/databases/0_counts.pkl', 
              '/data1/kji/databases/01_counts.pkl',
              '/data1/kji/databases/012_counts.pkl',
              '/data1/kji/databases/0123_counts.pkl']

In [6]:
for filename in test_files:
    with open(filename, "rb") as f:
        d = pickle.load(f)
        df_dict = defaultdict(list)
        combination = filename.split('/')[-1].split("_")[0]
        for n_gram, count in d.items():
            df_dict['combination'].append(combination)
            df_dict['n_gram'].append(str(n_gram))
            df_dict['matches'].append(count)
            df_dict['utility'].append(utilities[combination])
            df_dict['used'].append(0)
        d.clear()
        df = pd.DataFrame.from_dict(df_dict)
        with engine.begin() as connection:
            df.to_sql(combination, con=connection, index=False, if_exists='replace')
        print(f"finished processing {filename}")
        f.flush()
        df_dict.clear()

finished processing /data1/kji/databases/0_counts.pkl
finished processing /data1/kji/databases/01_counts.pkl
finished processing /data1/kji/databases/012_counts.pkl
finished processing /data1/kji/databases/0123_counts.pkl


### Generate database construction plan

In [4]:
def marginal_memory(num_matches, remaining_budget):
    return num_matches / remaining_budget

In [5]:
def marginal_runtime(matches_squared, num_matches, total_matches, total_avg_runtime, runtime_budget):
    cost = (matches_squared + num_matches ** 2) / (total_matches + num_matches) - total_avg_runtime
    return cost / (runtime_budget - total_avg_runtime)

In [6]:
def metric(utility, num_matches, memory_budget, runtime_budget, total_matches, matches_squared, total_avg_runtime):
    marginal_memory_cost = marginal_memory(num_matches, memory_budget-total_matches)
    marginal_runtime_cost = marginal_runtime(matches_squared, num_matches, total_matches, total_avg_runtime, runtime_budget)
    return utility / max(marginal_memory_cost, marginal_runtime_cost)

In [14]:
conn = sqlite3.connect("/data1/kji/databases/test.db")
conn.create_function("metric", 7, metric)
c = conn.cursor()

In [10]:
db_items = ['(2048, 0, 0, 0, 0, 0)', '(1073741824, 0, 0, 0, 0, 0)']

In [11]:
query = "select * from fingerprints where used == 0 order by utility desc limit 1"

In [15]:
result = c.execute(query).fetchone()

In [7]:
def query(db_path, table_name, memory_budget, runtime_budget, total_matches, matches_squared, total_avg_runtime):
    with closing(sqlite3.connect(db_path)) as con, con,  \
            closing(con.cursor()) as cur:
        con.create_function("metric", 7, metric)
        query = f"select * from '{table_name}' where used = 0 order by metric(utility, matches, ?, ?, ?, ?, ?) desc limit 1"
        cur.execute(query, (memory_budget, runtime_budget, total_matches, matches_squared, total_avg_runtime))
        result = cur.fetchone()
        ratio = metric(result[3], result[2], memory_budget, runtime_budget, total_matches, matches_squared, total_avg_runtime)
        return (result, ratio)

In [8]:
def construct_db(db_path, memory_budget, runtime_budget, outdir):
    """
    Inputs: a database which contains n-grams, their utility, and number of matches,
            a memory budget, and a runtime budget
    Output: a file specifying an ordered list of n-grams to include in the final database
    """
    conn = sqlite3.connect(db_path)
    
    with closing(conn.cursor()) as c:
        c.execute("SELECT name FROM sqlite_master WHERE type='table'")
        tables = c.fetchall()
        
    # clear all used bits in the database
    for table in tables:
        with closing(conn.cursor()) as cur:
            cur.execute(f"UPDATE '{table[0]}' SET used = 0")
    
    total_matches = 0
    matches_squared = 0
    total_avg_runtime = 0
    with open(f"{outdir}/Bm_10k_Bt_8k_fingerprints.txt", "w") as f1, open(f"{outdir}/Bm_10k_Bt_8k_info.csv", "w") as f2:
        writer = csv.writer(f2)
        writer.writerow(["fingerprint", "ratio", "m_i", "Ct_i", "Bm-Dm", "Bt-Dt"])
        while memory_budget - total_matches > 0 and runtime_budget - total_avg_runtime > 0:
            start = time.time()
            inputs = [(db_path, table[0], memory_budget, runtime_budget, total_matches, matches_squared, total_avg_runtime) for table in tables]
            with multiprocessing.Pool(processes = 26) as pool:
                results = pool.starmap(query, inputs)
            # get the pair with the highest ratio
            result = max(results, key = lambda pair: pair[1])
            n_gram = result[0][0]
            f1.write(f"{n_gram}\n")
            ratio = result[1]
            m_i = result[0][2]
            cost = (matches_squared + m_i ** 2) / (total_matches + m_i) - total_avg_runtime
            remaining_memory = memory_budget - total_matches
            remaining_runtime = runtime_budget - total_avg_runtime
            print(remaining_memory)
            print(remaining_runtime)
            writer.writerow([result[0][1], ratio, m_i, cost, remaining_memory, remaining_runtime])
            total_matches += m_i
            matches_squared += m_i**2
            total_avg_runtime += matches_squared / total_matches  
            combination = result[0][0]
            with closing(conn.cursor()) as c:
                c.execute(f"UPDATE '{combination}' SET used = 1 WHERE combination = '{combination}' and n_gram = '{n_gram}'")
            print(f"finished in {time.time() - start} seconds")

In [9]:
db_path = "/data1/kji/databases/test.db"

In [10]:
memory_budget = 10000

In [11]:
runtime_budget = 8000

In [11]:
%load_ext line_profiler

No multithreading takes 166 seconds per fingerprint.

In [21]:
%lprun -f construct_db construct_db(db_path, memory_budget, runtime_budget, "experiments/db_tests")

UsageError: Line magic function `%lprun` not found.


In [12]:
construct_db(db_path, memory_budget, runtime_budget, "experiments/db_tests")

10000
8000
finished in 49.985451221466064 seconds
9999
7999.0
finished in 46.40796089172363 seconds
9998
7998.0
finished in 44.28696370124817 seconds
9997
7997.0
finished in 46.907904386520386 seconds
9996
7996.0
finished in 46.40547823905945 seconds
9995
7995.0
finished in 52.72178411483765 seconds
9994
7994.0
finished in 45.79409456253052 seconds
9993
7993.0
finished in 46.2898633480072 seconds
9992
7992.0
finished in 46.557454109191895 seconds
9991
7991.0
finished in 44.22396373748779 seconds
9990
7990.0
finished in 49.21182632446289 seconds


Process ForkPoolWorker-311:
Process ForkPoolWorker-308:
Process ForkPoolWorker-297:
Process ForkPoolWorker-293:
Process ForkPoolWorker-310:
Process ForkPoolWorker-309:
Process ForkPoolWorker-304:
Process ForkPoolWorker-291:
Process ForkPoolWorker-305:
Process ForkPoolWorker-296:
Process ForkPoolWorker-307:
Process ForkPoolWorker-312:
Process ForkPoolWorker-306:
Process ForkPoolWorker-300:
Process ForkPoolWorker-298:
Process ForkPoolWorker-294:
Process ForkPoolWorker-299:
Process ForkPoolWorker-301:
Process ForkPoolWorker-303:


KeyboardInterrupt: 